<a href="https://colab.research.google.com/github/imallag/solana/blob/main/vwap_binance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ccxt


In [2]:
import ccxt
import pandas as pd
from datetime import datetime, timedelta

# Initialize Binance exchange
binance = ccxt.binance()

# Define the trading pair and date
symbol = 'ETH/USDT'
date = '2023-01-01'

In [3]:
# Convert date to timestamp
start_time = binance.parse8601(date + 'T00:00:00Z')
end_time = start_time + 24 * 60 * 60 * 1000


In [4]:
# Fetch trades
trades = []
since = start_time

while since < end_time:
    try:
        # Fetch trade data
        data = binance.fetch_trades(symbol, since)
        if len(data) == 0:
            break
        trades.extend(data)
        since = data[-1]['timestamp'] + 1
    except Exception as e:
        print(f"Error: {e}")
        break

In [ ]:
trades

In [6]:
# Convert trades to DataFrame
trades_df = pd.DataFrame(trades)

In [8]:
trades_df

,info,timestamp,datetime,symbol,id,order,type,side,takerOrMaker,price,amount,cost,fee,fees
0,"{'a': '826763415', 'p': '1196.13000000', 'q': ...",1672531200000,2023-01-01T00:00:00.000Z,ETH/USDT,826763415,None,None,sell,None,1196.13,0.0104,12.439752,None,[]
1,"{'a': '826763416', 'p': '1196.13000000', 'q': ...",1672531200007,2023-01-01T00:00:00.007Z,ETH/USDT,826763416,None,None,sell,None,1196.13,0.0119,14.233947,None,[]
2,"{'a': '826763417', 'p': '1196.13000000', 'q': ...",1672531200008,2023-01-01T00:00:00.008Z,ETH/USDT,826763417,None,None,sell,None,1196.13,0.0557,66.624441,None,[]
3,"{'a': '826763418', 'p': '1196.13000000', 'q': ...",1672531200008,2023-01-01T00:00:00.008Z,ETH/USDT,826763418,None,None,sell,None,1196.13,0.0150,17.941950,None,[]
4,"{'a': '826763419', 'p': '1196.14000000', 'q': ...",1672531200009,2023-01-01T00:00:00.009Z,ETH/USDT,826763419,None,None,buy,None,1196.14,0.0204,24.401256,None,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186495,"{'a': '826949994', 'p': '1200.72000000', 'q': ...",1672617720031,2023-01-02T00:02:00.031Z,ETH/USDT,826949994,None,None,buy,None,1200.72,0.0191,22.933752,None,[]
186496,"{'a': '826949995', 'p': '1200.72000000', 'q': ...",1672617720034,2023-01-02T00:02:00.034Z,ETH/USDT,826949995,None,None,buy,None,1200.72,0.0176,21.132672,None,[]
186497,"{'a': '826949996', 'p': '1200.72000000', 'q': ...",1672617720035,2023-01-02T00:02:00.035Z,ETH/USDT,826949996,None,None,buy,None,1200.72,0.0368,44.186496,None,[]
186498,"{'a': '826949997', 'p': '1200.72000000', 'q': ...",1672617720043,2023-01-02T00:02:00.043Z,ETH/USDT,826949997,None,None,buy,None,1200.72,0.0614,73.724208,None,[]


In [10]:
# Filter trades by date (if necessary)
trades_df['datetime'] = pd.to_datetime(trades_df['timestamp'], unit='ms')
trades_df = trades_df[(trades_df['datetime'] >= date) & (trades_df['datetime'] < (datetime.strptime(date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d'))]


In [11]:
# Calculate the weighted average price
trades_df['amount_usdt'] = trades_df['price'] * trades_df['amount']
weighted_avg_price = trades_df['amount_usdt'].sum() / trades_df['amount'].sum()

<ipython-input-11-d5e5fe255748>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trades_df['amount_usdt'] = trades_df['price'] * trades_df['amount']


In [14]:
trades_df

,info,timestamp,datetime,symbol,id,order,type,side,takerOrMaker,price,amount,cost,fee,fees,amount_usdt
0,"{'a': '826763415', 'p': '1196.13000000', 'q': ...",1672531200000,2023-01-01 00:00:00.000,ETH/USDT,826763415,None,None,sell,None,1196.13,0.0104,12.439752,None,[],12.439752
1,"{'a': '826763416', 'p': '1196.13000000', 'q': ...",1672531200007,2023-01-01 00:00:00.007,ETH/USDT,826763416,None,None,sell,None,1196.13,0.0119,14.233947,None,[],14.233947
2,"{'a': '826763417', 'p': '1196.13000000', 'q': ...",1672531200008,2023-01-01 00:00:00.008,ETH/USDT,826763417,None,None,sell,None,1196.13,0.0557,66.624441,None,[],66.624441
3,"{'a': '826763418', 'p': '1196.13000000', 'q': ...",1672531200008,2023-01-01 00:00:00.008,ETH/USDT,826763418,None,None,sell,None,1196.13,0.0150,17.941950,None,[],17.941950
4,"{'a': '826763419', 'p': '1196.14000000', 'q': ...",1672531200009,2023-01-01 00:00:00.009,ETH/USDT,826763419,None,None,buy,None,1196.14,0.0204,24.401256,None,[],24.401256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186210,"{'a': '826949709', 'p': '1200.33000000', 'q': ...",1672617599996,2023-01-01 23:59:59.996,ETH/USDT,826949709,None,None,sell,None,1200.33,0.0444,53.294652,None,[],53.294652
186211,"{'a': '826949710', 'p': '1200.33000000', 'q': ...",1672617599997,2023-01-01 23:59:59.997,ETH/USDT,826949710,None,None,sell,None,1200.33,0.0114,13.683762,None,[],13.683762
186212,"{'a': '826949711', 'p': '1200.33000000', 'q': ...",1672617599998,2023-01-01 23:59:59.998,ETH/USDT,826949711,None,None,sell,None,1200.33,0.0142,17.044686,None,[],17.044686
186213,"{'a': '826949712', 'p': '1200.34000000', 'q': ...",1672617599998,2023-01-01 23:59:59.998,ETH/USDT,826949712,None,None,buy,None,1200.34,0.0094,11.283196,None,[],11.283196


In [15]:
weighted_avg_price2 = trades_df['cost'].sum() / trades_df['amount'].sum()

In [12]:
# Print the result
print(f"Weighted Average Price: {weighted_avg_price:.8f}")
print(f"Sum of first 8 digits after comma: {sum(int(digit) for digit in str(weighted_avg_price).split('.')[1][:8])}")

Weighted Average Price: 1197.45858801
Sum of first 8 digits after comma: 38


In [16]:
# Print the result
print(f"Weighted Average Price: {weighted_avg_price2:.8f}")
print(f"Sum of first 8 digits after comma: {sum(int(digit) for digit in str(weighted_avg_price2).split('.')[1][:8])}")

Weighted Average Price: 1197.45858801
Sum of first 8 digits after comma: 38
